## Data Visualization

In [1]:
!pip install pyspark -I

Processing c:\users\aksha\appdata\local\pip\cache\wheels\0b\90\c0\01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f\pyspark-3.1.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/9e/b6/6a4fb90cd235dc8e265a6a2067f2a2c99f0d91787f06aca4bcf7c23f3f80/py4j-0.10.9-py2.py3-none-any.whl


In [2]:
# Miscellaneous
import random

# Numerical calculation
import numpy as np

# Data Handling
import pandas as pd

# Spark
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession

# Data Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import cufflinks as cf
import plotly as py
import plotly.graph_objs as go
import plotly.express as px
from plotly.offline import download_plotlyjs,init_notebook_mode,plot,iplot

### Report generation
This piece of calculations are being done with Spark and inserted into a partitioned Reports table in Hive

Spark job generates a CSV file from Hive partitioned table

In [3]:
cc = pd.read_csv('CCDMS_dataset.csv')
points_table = {k: v for k, v in zip(cc.purchase_category.unique(), [0.8,1.7,0.85,3,1.1,1.5,2.1,1.3,2,1.5,1.01,2.2,1.5,1])}
cc['reward_points'] = cc.apply(lambda row: points_table[row['purchase_category']] * row['trans_amt'], axis=1)
cc['cust_name'] = cc.apply(lambda row: f"{row['first']} {row['last']}", axis=1)
cc.trans_date_trans_time = pd.to_datetime(cc.trans_date_trans_time)
cc['trans_month'] = cc.trans_date_trans_time.apply(lambda dt: f"{dt.year}-{str(dt.month).zfill(2)}")
report = cc.groupby(['card_num', 'trans_month', 'purchase_category']) \
                                .agg(dict(cust_name='first', trans_id='count', trans_amt='sum', reward_points='sum')) \
                                .rename(columns={'trans_id': 'trans_count'}) \
                                .reset_index()
report.to_csv('report.csv', index=False)
report.head()

,card_num,trans_month,purchase_category,cust_name,trans_count,trans_amt,reward_points
0,60416207185,2020-06,entertainment,Mary Diaz,1,102.15,204.300
1,60416207185,2020-06,food,Mary Diaz,1,41.99,88.179
2,60416207185,2020-06,gas_transport,Mary Diaz,7,492.71,739.065
3,60416207185,2020-06,grocery_net,Mary Diaz,1,63.12,63.120
4,60416207185,2020-06,grocery_pos,Mary Diaz,3,277.13,609.686


### <u>Data Load</u>
Load the Hive table into Pandas dataframe

In [4]:
# Set Env variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.7-bin-hadoop2.7"

# Create Spark Context
conf = (SparkConf().setMaster("local").setAppName("CCDMS"))
sc = SparkContext(conf=conf)
spark = SparkSession \
    .builder \
    .appName("CCDMS Visualizations") \
    .config("spark.sql.warehouse.dir", "file://tmp") \
    .getOrCreate()

# Load the data
report.head()

,card_num,trans_month,purchase_category,cust_name,trans_count,trans_amt,reward_points
0,60416207185,2020-06,entertainment,Mary Diaz,1,102.15,204.300
1,60416207185,2020-06,food,Mary Diaz,1,41.99,88.179
2,60416207185,2020-06,gas_transport,Mary Diaz,7,492.71,739.065
3,60416207185,2020-06,grocery_net,Mary Diaz,1,63.12,63.120
4,60416207185,2020-06,grocery_pos,Mary Diaz,3,277.13,609.686


Load the generated csv file by Spark job

In [5]:
# Python starts reading the generated csv file by Spark job and creates visualizations
report = pd.read_csv('report.csv')

### <u>Exploratory Data Analysis</u>
Exploratory Data Analysis (EDA) is an approach/philosophy for data analysis that employs a variety of techniques (mostly graphical) to
- maximize insight into a data set;
- uncover underlying structure;
- extract important variables;
- detect outliers and anomalies;
- test underlying assumptions;
- develop parsimonious models; and
- determine optimal factor settings

We'll be using plotly library to generate the graphs and visualizations. We need [cufflinks](https://plot.ly/python/v3/ipython-notebooks/cufflinks/) to link plotly to pandas dataframe and add the iplot method

Installation:<br/>
using pypi: **`!pip install plotly cufflinks`**<br/>
using conda: **`conda install -c conda-forge plotly cufflinks`**

#### 1. Number of transactions in each Purchase category
Find out how many transactions are recorded under each purchase category in the entire dataset

In [6]:
txncnt_grp = report.groupby('purchase_category')['trans_count'] \
                            .sum() \
                            .reset_index() \
                            .sort_values(by='trans_count', ascending=False)

# Pie chart
txncnt_grp.iplot(
    kind='pie', 
    labels='purchase_category', 
    values='trans_count', 
    title='Purchase Category- Pie Chart', 
    hoverinfo="label+percent+name", hole=0.25,
    colorscale='spectral',
    pull=np.linspace(0,0.2,report.purchase_category.nunique()))

#### 2. Amount spent in each Purchase category
Find out how much cumulative amount is spent for each purchase category in the entire dataset

In [7]:
txnamt_grp = report.groupby('purchase_category')['trans_amt'].sum().reset_index()

# Bar plot
txnamt_grp.iplot(
    kind='bar', 
    x="purchase_category", 
    y="trans_amt",
    xTitle='Purchase Category',
    yTitle='Amount',
    colorscale='-plotly',
    title='Transaction Amount Distribution- Bar Chart')

#### 3. Monthly amount spent in each Purchase category
Find out month wise expenditures for each purchase category in the entire dataset

In [8]:
txnamt_mnth_grp = report.groupby(['trans_month', 'purchase_category'])['trans_amt'].sum().reset_index()
px.bar(
    data_frame=txnamt_mnth_grp, 
    x="trans_month", 
    y="trans_amt", 
    color="purchase_category",
    labels=dict(trans_month="Transaction Month", trans_amt="Amount"),
    title="Amount spent on Purchase Category month wise - Stacked Bar")

#### 4. Monthly transaction counts
Find out month wise recorded transaction counts

In [9]:
txncntmnt_grp = report.groupby('trans_month')['trans_count'].sum().reset_index()

# Pie chart
txncntmnt_grp.iplot(
    kind='line', 
    x="trans_month", 
    y="trans_count",
    xTitle='Month',
    yTitle='Number of Transactions',
    title='Transaction Count Distribution- Line Plot')

### Let's find out visualizations for a random customer
#### 5. Monthly reward points earned in each Purchase category
Find out month wise reward points earned for each purchase category for a random customer

In [10]:
cust = report[report.card_num == random.choice(cc.card_num)]
cust_txnamt_mnth_grp = cust.groupby(['trans_month', 'purchase_category'])['reward_points'].sum().reset_index()
px.bar(
    data_frame=cust_txnamt_mnth_grp, 
    x="trans_month", 
    y="reward_points", 
    color="purchase_category",
    labels=dict(trans_month="Transaction Month", reward_points="Reward Points earned"),
    title="Amount spent on Purchase Category month wise - Stacked Bar")

#### 6. Amount spent in each month
Find out total amount spent with credit card transactions for a random customer in each month

In [11]:
cust_txnamt_mnth_grp = cust.groupby('trans_month')['trans_amt'].sum().reset_index()
# Bar plot
cust_txnamt_mnth_grp.iplot(
    kind='bar', 
    x="trans_month", 
    y="trans_amt",
    xTitle='Transaction Month',
    yTitle='Amount',
    colorscale='-GnBu',
    title='Transaction Amount Distribution- Bar Chart')

#### 7. Number of transactions in purchase category
Find out number of credit card transactions for a random customer in purchase category

In [12]:
cust_txncnt_mnth_grp = cust.groupby('purchase_category')['trans_count'] \
                            .sum() \
                            .reset_index()

# Pie chart
cust_txncnt_mnth_grp.iplot(
    kind='pie', 
    labels='purchase_category', 
    values='trans_count', 
    title='Purchase Category- Pie Chart', 
    hoverinfo="label+percent+name",
    colorscale='spectral',
    pull=np.linspace(0,0.2,report.purchase_category.nunique()))